Bibliotecas

In [ ]:
import pdfplumber
import os
import re
import unidecode

In [ ]:
# Definir informações universais 
caminho_pdf_selecionado = "../data/input/RCG2025.pdf"
caminho_data_input="../data/input/"

Extração de texto bruto: remover cabeçalhos, rodapés, numeração de páginas.

In [ ]:
# Abrir pdfs e extrair texto bruto
if os.path.exists(caminho_pdf_selecionado):
    print(f"Arquivo encontrado: {caminho_pdf_selecionado}")
    texto_bruto = ""
    with pdfplumber.open(caminho_pdf_selecionado) as pdf:
        print(f"Total de páginas: {len(pdf.pages)}")
        for i, pagina in enumerate(pdf.pages):
            print(f"Processando página {i+1}...")
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                texto_bruto += texto_pagina + "\n"
else:
    print(f"ERRO: Arquivo não encontrado: {caminho_pdf_selecionado}")
    print("Arquivos disponíveis:")
    for arquivo in os.listdir(caminho_data_input):
        print(f"  - {arquivo}")  

Normalização linguística: padronizar texto e expandir siglas.

In [ ]:
# tudo minúsculo
texto_limpo = texto_bruto.lower()  
# remove quebras de linha repetidas
texto_limpo = re.sub(r'\n+', ' ', texto_limpo)  
# remove espaços extras
texto_limpo = re.sub(r'\s+', ' ', texto_limpo)  
# remove números
texto_limpo = re.sub(r'\d+', '', texto_limpo) 
# remove pontuação 
texto_limpo = re.sub(r'[^\w\s]', '', texto_limpo)   

In [ ]:
# remove acentos
texto_normalizado = unidecode.unidecode(texto_limpo)  
texto_normalizado = texto_normalizado.strip()

Expansão de siglas e abreviações

In [ ]:
# Dicionário de substituições
substituicoes = {
    "tcc": "trabalho de conclusão de curso",
    "sgbd": "sistema de gerenciamento de banco de dados"
}

In [ ]:
def expandir_siglas(texto, substituicoes):
    for sigla, expansao in substituicoes.items():
        # \b garante que a troca ocorre apenas em palavras inteiras
        texto = re.sub(rf'\b{sigla}\b', expansao, texto)
    return texto

texto_normalizado = expandir_siglas(texto_normalizado, substituicoes)

Detecção de estrutura: identificar capítulos, seções, artigos, parágrafos, incisos.

In [ ]:
padroes = {
    "capitulo": r'cap[ií]tulo\s+[ivxlcdm]+',
    "secao": r'se[cç][aã]o\s+[ivxlcdm]+',
    # "artigo": r'art\.?\s*\d+[º°]?',
    # "paragrafo": r'§\s*\d+[º°]?',
    # "inciso": r'^[ivxlcdm]+\s*[-–)]',
}

for tipo, regex in padroes.items():
    resultados = re.findall(regex, texto_normalizado, flags=re.IGNORECASE | re.MULTILINE)
    print(f"{tipo.title()}: {resultados}")

Extração de tabelas: converter tabelas do PDF em formato estruturado (linhas e colunas).

In [ ]:
import pdfplumber
import pandas as pd

caminho_pdf = "../data/input/RCG2025.pdf"
tabelas_extraidas = []

with pdfplumber.open(caminho_pdf) as pdf:
    for pagina in pdf.pages:
        tabelas = pagina.extract_tables()
        for tabela in tabelas:
            df = pd.DataFrame(tabela[1:], columns=tabela[0])  # primeira linha = cabeçalho
            tabelas_extraidas.append(df)

# Exemplo: visualizar a primeira tabela
print(tabelas_extraidas[0].head())

# Salvar em CSV
tabelas_extraidas[0].to_csv("tabela1.csv", index=False)

In [ ]:
import pdfplumber
import pandas as pd

tabelas_extraidas = []

with pdfplumber.open("../data/input/RCG2025.pdf") as pdf:
    for i, pagina in enumerate(pdf.pages, start=1):
        tabelas = pagina.extract_tables()
        for t in tabelas:
            df = pd.DataFrame(t[1:], columns=t[0])  # primeira linha = cabeçalho
            tabelas_extraidas.append(df)
            print(f"Tabela encontrada na página {i}:")
            print(df.head(), "\n")

In [ ]:
print(f"\nTexto extraído ({len(texto_normalizado)} caracteres):")
print(texto_normalizado[:1000])

Deduplicação e consistência: eliminar trechos redundantes ou repetidos.

In [ ]:
# Deduplicação de linhas idênticas
linhas = texto_normalizado.splitlines()
linhas_unicas = list(dict.fromkeys(linhas))  # mantém ordem, remove duplicadas
texto_deduplicado = "\n".join(linhas_unicas)

# Deduplicação aproximada (similaridade textual)
import difflib

linhas_unicas = []
for linha in linhas:
    if not linhas_unicas or difflib.SequenceMatcher(None, linha, linhas_unicas[-1]).ratio() < 0.9:
        linhas_unicas.append(linha)

texto_deduplicado = "\n".join(linhas_unicas)

# Remoção de cabeçalhos e rodapés automáticos
# Usar expressões regulares para eliminar padrões previsíveis:
texto_deduplicado = re.sub(r'p[áa]gina\s*\d+\s*(de\s*\d+)?', '', texto_deduplicado, flags=re.IGNORECASE)
texto_deduplicado = re.sub(r'minist[eé]rio da educa[cç][aã]o', '', texto_deduplicado, flags=re.IGNORECASE)

# Normalização de consistência textual
texto_final = re.sub(r'\s+', ' ', texto_deduplicado).strip()

# Deduplicação em nível de parágrafo
paragrafos = [p.strip() for p in texto_deduplicado.split('\n\n') if p.strip()]
paragrafos_unicos = list(dict.fromkeys(paragrafos))
texto_final = '\n\n'.join(paragrafos_unicos)

Enriquecimento com metadados: acrescentar informações como doc_id, nome_doc, versão, data_publicação, páginas inicial e final.

In [ ]:
# Extração automática de metadados do PDF 
import pdfplumber
from pathlib import Path

arquivo = Path("documento.pdf")

with pdfplumber.open(arquivo) as pdf:
    info_pdf = pdf.metadata
    num_paginas = len(pdf.pages)

print(info_pdf)

# Criar o dicionário de metadados
from datetime import datetime

metadados = {
    "doc_id": "DOC_001",
    "nome_doc": arquivo.name,
    "versao": "1.0",
    "data_publicacao": datetime.strptime("2024-04-10", "%Y-%m-%d").date(),
    "paginas_inicial": 1,
    "paginas_final": num_paginas,
    "autor": info_pdf.get("/Author"),
    "titulo": info_pdf.get("/Title"),
}

# Combinar texto + metadados
documento = {
    "metadados": metadados,
    "texto_limpo": texto_final
}

import json

with open("documento_processado.json", "w", encoding="utf-8") as f:
    json.dump(documento, f, ensure_ascii=False, indent=2)

# Se estiver processando vários PDFs
# Montar uma lista de documentos enriquecidos
import glob

todos_docs = []

for caminho in glob.glob("pdfs/*.pdf"):
    with pdfplumber.open(caminho) as pdf:
        texto = "\n".join(pagina.extract_text() for pagina in pdf.pages if pagina.extract_text())
        doc = {
            "metadados": {
                "doc_id": Path(caminho).stem.upper(),
                "nome_doc": Path(caminho).name,
                "versao": "1.0",
                "data_publicacao": None,  # pode ser extraída depois
                "paginas_inicial": 1,
                "paginas_final": len(pdf.pages),
            },
            "texto_limpo": texto
        }
        todos_docs.append(doc)

# Estrutura final sugerida (JSON)
{
  "doc_id": "DOC_001",
  "nome_doc": "relatorio_2024.pdf",
  "versao": "1.0",
  "data_publicacao": "2024-04-10",
  "paginas_inicial": 1,
  "paginas_final": 25,
  "autor": "Ministério da Economia",
  "titulo": "Relatório Anual 2024",
  "texto_limpo": "conteúdo limpo e deduplicado do PDF..."
}